In [2]:
# import libarary
import torch
import torch.nn.functional as F
import torch.nn as nn
import pandas as pd
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
# read the data
data = pd.read_csv('/kaggle/input/150k-lyrics-labeled-with-spotify-valence/labeled_lyrics_cleaned.csv', encoding = 'utf-8')['seq']
text = ''.join(data.tolist())

In [4]:
print(f'number of characters :  {len(text)}')

number of characters :  179890373


In [5]:
# make vocab
chars = sorted(list(set(text)))

#vocab karakter ke index
stoi = {c:i for i,c in enumerate(chars)}

#vocab index ke karakter
itos = {i:c for i,c in stoi.items()}

print(f'jumlah vocab :  {len(stoi)}')
print(''.join(chars))

jumlah vocab :  99
	
 !"#$%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\]^_`abcdefghijklmnopqrstuvwxyz{|}~


In [6]:
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}

# fungsi encode dan decode yang mengembalikan list index atau list karakter dari input
encode = lambda x : [stoi[c] for c in x]
decode = lambda x : ''.join([itos[i] for i in x])


print(encode('Hi Yubaba'))
print(decode(encode('Hi Yubaba')))

[43, 76, 3, 60, 88, 69, 68, 69, 68]
Hi Yubaba


In [7]:
# encode entire text then store it into a tensor
data = torch.tensor(encode(text), dtype = torch.long)
print(data.shape, data.dtype)

torch.Size([179890373]) torch.int64


In [8]:
# split train and val dataset
n = int(0.9 * len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

print(train_data.shape)
print(val_data.shape)

torch.Size([161901335])
torch.Size([17989038])


In [9]:
block_size = 8
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f'when input is {context} the target : {target}')

when input is tensor([49]) the target : 82
when input is tensor([49, 82]) the target : 15
when input is tensor([49, 82, 15]) the target : 3
when input is tensor([49, 82, 15,  3]) the target : 81
when input is tensor([49, 82, 15,  3, 81]) the target : 82
when input is tensor([49, 82, 15,  3, 81, 82]) the target : 2
when input is tensor([49, 82, 15,  3, 81, 82,  2]) the target : 1
when input is tensor([49, 82, 15,  3, 81, 82,  2,  1]) the target : 44


In [10]:
torch.manual_seed(1337)

def get_batches(split):
    # generate small batch of data input x and y
    if split == 'train':
        data = train_data
    else:
        data = val_data

    ix = torch.randint(low = 0, high = len(data) - block_size, size = (batch_size,)) #generate a tensor contains random integer, used as first index for 1 context sample
    x = torch.stack([data[i:i+block_size] for i in ix],)
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x , y = x.to(device), y.to(device)
    return x, y

In [11]:
batch_size = 16

xb, yb = get_batches('train')
print(xb)
print(yb)
print(xb.shape)
print(yb.shape)

tensor([[82, 88, 74, 75,  3, 87, 75, 72],
        [44, 10, 71,  3, 78, 76, 86, 86],
        [72, 85, 73, 72, 70, 87,  3, 86],
        [87,  4, 12,  2,  1, 47, 72, 87],
        [88, 71, 15,  3, 69, 72, 70, 68],
        [10, 87,  3, 90, 68, 76, 87,  2],
        [82, 90, 81,  3, 34,  5,  2,  1],
        [ 3, 71, 82, 90, 81,  3, 76, 81],
        [42, 85, 72, 74, 82, 85, 92, 15],
        [71,  3, 69, 76, 74,  3, 82, 79],
        [86, 87,  3, 75, 68, 86,  3, 85],
        [ 3, 81, 76, 74, 75, 87, 15,  3],
        [82, 88,  3, 80, 68, 78, 72,  3],
        [10, 89, 72,  3, 78, 76, 86, 86],
        [87, 75, 72,  3, 73, 68, 80, 72],
        [88, 85,  3, 92, 72, 68, 85, 86]], device='cuda:0')
tensor([[88, 74, 75,  3, 87, 75, 72,  3],
        [10, 71,  3, 78, 76, 86, 86,  3],
        [85, 73, 72, 70, 87,  3, 86, 87],
        [ 4, 12,  2,  1, 47, 72, 87, 10],
        [71, 15,  3, 69, 72, 70, 68, 88],
        [87,  3, 90, 68, 76, 87,  2,  1],
        [90, 81,  3, 34,  5,  2,  1,  2],
        [71, 82,

In [12]:
block_size = 3 # berapa karakter pendahulu yang dipakai untuk memprediksi karakter setelahnya

#create training set (x,y)
def build_dataset(words):
    X, Y = [], []
    for w in words:
        #print(w)
        context = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            #print(''.join(itos[i] for i in context) , ' ---> ',itos[ix])
            context = context[1:] + [ix]
    X = torch.tensor(X, device = device)
    Y = torch.tensor(Y, device = device)
    return X,Y

In [13]:

vocab_size = len(chars)


#hyperparamaters
batch_size = 64 # how many independent sequence will be process in parallel
block_size = 256 # maximum context length
max_iters = 20000
eval_interval = 500
learning_rate = 3e-4
eval_iters = 200
n_emb = 384
n_head = 6
n_layer = 6
dropout = 0.2

In [14]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X,Y = get_batches(split)
            logits, loss = model(X,Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [15]:
import torch
import torch.nn as nn
from torch.nn import functional as F


torch.manual_seed(1337)



class MultiHeadAttention(nn.Module):
    
    '''ini merupakan implementasi multihead attention tanpa harus membuat class head dan multihead secara terpisah
    tetapi dimensi head nya dimasukkan seperti batch pada tensornya'''
    
    def __init__(self,n_head, head_size):
        super().__init__()
        # membuat layer key, query, value dalam 1 linear layer
        self.kqv = nn.Linear(n_emb, n_emb*3, bias = False) # n_emb*3 karena kita ingin membuat linear layer untuk 3 tensor k, q, v sekaligus
        self.register_buffer('tril', torch.tril(torch.ones([block_size, block_size]).view(1,1,block_size,block_size))) # pada pytorch kita assign tril seperti ini agar dianggap bukan parameter
        self.attn_dropout = nn.Dropout(dropout)  # attention layer dropout
        self.res_dropout = nn.Dropout(dropout)  # residual layer dropout
        self.proj = nn.Linear(n_emb, n_emb) #projection layer (lihat papernya)
        
    def forward(self,x):
        B,T,C = x.shape
        k, q, v = self.kqv(x).split(n_emb, dim=2)
        
        k = k.view(B,T,n_head, C//n_head).transpose(1,2) #(B,n_head,T,head_size)
        q = q.view(B,T,n_head, C//n_head).transpose(1,2) #(B,n_head,T,head_size)
        v = v.view(B,T,n_head, C//n_head).transpose(1,2) #(B,n_head,T,head_size)
        
        attn = q @ k.transpose(-2,-1) * (k.shape[-1]**-0.5) # (B,n_head,T, head_size) @ (B,n_head, head_size,T) --> (B,n_head,T,T)
        attn = attn.masked_fill(self.tril[:,:,:T, :T] == 0, float('-inf')) # (B,n_head,T,T)
        attn = F.softmax(attn, dim=-1) # (B,n_head,T,T)
        attn= self.attn_dropout(attn) #(B,n_head,T,T)

        #perform weighted aggregation of v
        out = attn @ v  #(B,n_head,T,T) @ (B,n_head,T,head_size) --> (B,n_head,T,head_size)
        out = out.transpose(1,2).contiguous().view(B,T,n_emb)  # menumpuk/concat head size dan n_head di dimensi terakhir menjadi (B,T,head_size*n_head)
        out = self.res_dropout(self.proj(out))
        return out
        
        


class FeedForward(nn.Module):
    '''simple linear layer followed by a non linearity
    lihat paper pada bagian : 3.3 Position-wise Feed-Forward Networks'''

    def __init__(self,n_emb):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_emb, n_emb*4), # pada paper inner layernya lebih besar 4 kali (dari 512 -> 2048) kita coba menirunya
            nn.ReLU(),
            nn.Linear(n_emb*4, n_emb), #projection layer setelah feedforward,
            nn.Dropout(dropout)
        )

    def forward(self,x):
        return self.net(x)



class Block(nn.Module):
    '''Transformer block : communication (attention layer) followed by computation(feedforward layer)'''
    def __init__(self, n_emb, n_head):
        super().__init__()
        head_size = n_emb//n_head  #kita mengurangi dimensi dari headnya sesuai dengan jumlah headnya, sehingga computation costnya sama saja dengan single head attention dengan full dimensi
        self.ma = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_emb)
        self.layernorm_1 = nn.LayerNorm(n_emb)
        self.layernorm_2 = nn.LayerNorm(n_emb)

    def forward(self,x):
        x = x + self.ma(self.layernorm_1(x))  #skip connection and layer norm before attention layer
        x = x + self.ffwd(self.layernorm_2(x)) #skip connection and layer norm before feedforward layer (lihat paper attention is all you need)
        return x



class LanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_emb)
        self.position_embedding_table = nn.Embedding(block_size, n_emb)
        self.blocks = nn.Sequential(*[Block(n_emb,n_head=n_head) for _ in range(n_layer)])
        self.lm_head = nn.Linear(n_emb, vocab_size)

    def forward(self, idx, targets = None):

        B,T = idx.shape

        token_emb = self.token_embedding_table(idx)  #(B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device = device))  #(T,C)
        x = token_emb + pos_emb # (B,T,C)
        x = self.blocks(x)
        logits = self.lm_head(x)  #(B,T,vocab_size)

        if targets == None:
            loss = None
        else:
            B, T, C = logits.shape  # (32,8,65)
            logits =  logits.view(B*T, C) #(32,65)
            targets = targets.view(B*T) # (32)
            loss = F.cross_entropy(logits,targets)

        return logits, loss


    def generate(self, idx, max_new_tokens):
        # idx : (B,T) tensor berisi index dari context dari current time step
        # max_new_tokens : jumlah max karakter yang ingin di generate
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:] #karena menggunakan positional embedding, idx harus dibatasi dengan shifting ketika panjang context sudah lebih dari block_size
            logits, loss = self(idx_cond)  # prediction
            logits = logits[:, -1, :] # ambil hanya time step terakhir menjadi (B,C)
            probs = F.softmax(logits, dim = -1) # softmax untuk mendapatkan distribusi probabilitas untuk setiap class
            idx_next = torch.multinomial(probs, num_samples=1) # abmil sampel berdasarkan distribusi probabilitas
            idx = torch.cat((idx, idx_next), dim = 1) # tempelkan ke karakter terakhir
        return idx

model = LanguageModel()
model = model.to(device)


logits,loss = model(xb,yb)
print(loss)

tensor(4.9741, device='cuda:0', grad_fn=<NllLossBackward0>)


In [16]:
optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate)

In [ ]:
for iter in range(max_iters+1):

    #evaluate the loss on train and val
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    #sample batch from data
    xb, yb = get_batches('train')

    #evaluate the loss
    logits, loss = model(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 4.8751, val loss 4.8734
step 500: train loss 1.9134, val loss 1.9172
step 1000: train loss 1.5578, val loss 1.5654
step 1500: train loss 1.3998, val loss 1.4081


In [ ]:
idx = torch.zeros((1,1), dtype = torch.long, device = device) #kick off the generation
print(decode(model.generate(idx, max_new_tokens = 2000)[0].tolist()))

In [ ]:
#save model
state = {
    vocab_size : 99,
    batch_size : 64, # how many independent sequence will be process in parallel
    block_size : 256, # maximum context length
    max_iters : 5000,
    eval_interval : 500,
    learning_rate : 3e-4,
    eval_iters : 200,
    n_emb : 384,
    n_head : 6,
    n_layer : 6,
    dropout : 0.2,
    optimizer : optimizer.state_dict(),
    model : model.state_dict()
}


torch.save(state, '/kaggle/working/GPT_lyrics_2_times_running.pth')